# Consistencia y Concurrencia en SQL

## Transacción: Primer Eslabón de la Consistencia

Una herramienta importante que debe tener todo motor de base de datos es la de garantizar la **consistencia** de los datos.

Para ello, una solución es definir el concepto de transacción:

"*Es la denominación atómica de un conjunto de acciones, resuelta en su completitud o no resulta. Se considera como una unidad básica de recuperación que garantiza la integridad de los datos*"





### Ciclo de Vida de una Transacción

El ciclo de vida de una transacción en SQL sigue estos pasos fundamentales:

1. **BEGIN**  
   La transacción comienza explícitamente con el comando `BEGIN` (o `START TRANSACTION`). A partir de aquí, todas las operaciones forman parte de una misma unidad lógica.

2. **Ejecución de Operaciones**  
   Se ejecutan una o varias instrucciones SQL (INSERT, UPDATE, DELETE, etc.) que modifican los datos.

3. **¿Ocurrió algún fallo?**  
   - **Sí, hubo un error:**  
     Se ejecuta un `ROLLBACK`, lo que deshace todos los cambios realizados durante la transacción. Los datos vuelven al estado anterior al `BEGIN`.
   - **No, todo salió bien:**  
     Se ejecuta un `COMMIT`, lo que confirma y guarda de forma permanente todos los cambios realizados en la transacción.

**Resumen visual:**
<center>
BEGIN 

↓

EJECUCIÓN DE OPERACIONES

↓

¿FALLO? 

SI↙ ↘NO 

ROLLBACK     COMMIT
</center>

## Recuperación, Logs y el Recovery Manager

La consistencia de los datos no solo se basa en transacciones. Si bien la transacción me sirve para mantener un conjunto de acciones atómica, existe otro concepto a tener en cuenta: **la consistencia o coherencia de datos que estén en un almacenamiento intermedio**.

Cuando hacemos un commit y por lo tanto damos como terminada una transacción, esto no significa que esta nueva información impacte directamente en la base de datos, porque puede que los cambios sigan en el caché o buffer de la RAM por temas de optimización.

> De hecho, al evento en el cuál el caché en memoria impacta en la base de datos real se lo denomina **Checkpoint** y es un tipo de operación.

Si llega a pasar que ante un error de sistema no hubo Checkpoint, estaremos en un problema. 

Por ello,la mayoría de los motores de bases de datos implementan un sistema "*directo a disco*" que almacena las acciones (no los datos) realizados a modo de historial, para que en caso de un fallo estas acciones se puedan repetir. Esto se conoce como Logging.







### Logs



Los logs son archivos que registran secuencialmente todas las operaciones que modifican los datos de una base de datos **en el almacenamiento persistente**. Su propósito es garantizar la recuperación y la consistencia de la información ante fallos del sistema o caídas inesperadas.

Cada entrada en el log (también conocida como registro de log) contiene información detallada sobre una operación específica, permitiendo reconstruir o revertir transacciones según sea necesario.

De esta forma, se almacena lo siguiente:
- **Log Sequence Number (LSN)**: Identificador único y secuencial de la entrada en el log.
- **LSN anterior**: Referencia al LSN del registro anterior, formando una cadena de dependencias (como una LLI).
- **Timestamp**: Momento exacto en que se realizó la operación.
- **ID de Transacción**: Identificador único de la transacción que ejecutó la operación (Más de un registro/operación por transacción).
- **LSN previo de la transacción**: Último LSN de la transacción anterior.
- **Acción realizada**: Tipo de operación ejecutada, como `insert`, `delete`, `update`, `begin`, `commit`, o `checkpoint`.
- **Item alterado**: Identificador del elemento de datos modificado.
- **Valor previo**: Valor que tenía el item antes de la operación.
- **Valor actual**: Valor que tiene el item luego de la operación.


### Recovery Manager y Algoritmo ante Fallas

Esta recuperación ante fallas que se presentó en la sección anterior corresponde con las competencias de un componente muy importante de cualquier motor de base de datos: el **Recovery Manager**.

El mismo es capaz de ejecutar rutinas de recuperación capaces de solucionar la integridad de los datos ante eventos desafortunados como los antes mencionados.

>**NOTA**:
>
>Los datos en "*peligro*" de los que se habla son aquellos ocurridos entre el último Checkpoint y la falla.

Una de las *rutinas* que ejecuta es lo que se conoce como "*Algoritmo Undo- Redo*"

#### Algoritmo Undo-Redo

Funciona de la siguiente manera:

0. Supongamos que se ejecutaron varias transacciones y se realizó un checkpoint. Luego, ocurrieron más transacciones hasta que se produjo una falla. Algunas de esas operaciones no se reflejaron en la base de datos, pero pueden recuperarse gracias al log y al Recovery Manager.

1. Se crean dos listas:
   - **UNDO**: contiene transacciones activas al momento del checkpoint.
   - **REDO**: almacenará transacciones que hayan finalizado correctamente.

2. Se recorre el log en orden cronológico:
   - Si se encuentra un `BEGIN`, se añade la transacción a la lista UNDO.
   - Si se encuentra un `COMMIT`, la transacción pasa de UNDO a REDO.

3. Al finalizar el recorrido, se procesan los logs en orden inverso:
   - Se **deshacen (UNDO)** las operaciones de transacciones incompletas (como rollback).
   - Se **rehacen (REDO)** las operaciones de transacciones confirmadas.






## Concurrencia y Serialización

### El Problema de la Concurrencia


Debido a que cada transacción realiza operaciones que implican variables intermedias, si se interlazan operaciones de varias transacciones concurrentes sin cuidado se puede llegar a tener condiciones de carrera indeseables capaces de reflejar inconsistencias en los datos.

>**Ejemplo clásico:**
>
>Supongamos que dos transacciones, T1 y T2, acceden al mismo saldo de una cuenta bancaria con $100 de balance.
>
>- T1 quiere **retirar $50**.
>- T2 quiere **retirar $70** al mismo tiempo.
>
>1. T1 lee el saldo: 100
>2. T2 lee el saldo: 100
>3. T1 escribe saldo = 100 - 50 = 50
>4. T2 escribe saldo = 100 - 70 = 30
>
>Resultado final: el saldo queda en **$30**, cuando en realidad deberían haberse retirado solo $50 *o* $70, no ambos. Se "pierde" una operación y el sistema queda en un estado inconsistente.




Casos como el ejemplo de arriba nos dicen que **hay maneras de ejecutar transacciones concurrentes tanto correcta como incorrectamente**.



### Supuestos de Correctiud en una Ejecución

Se supone que hubo ejecución correcta cuando se cumpla cualquiera de estos tres supuestos:
- Solo se esta ejecutando una sola transacción (medio trivial, ya sé)
- Se están ejecutando varias transacciones, pero de manera serial.
    $$ T1.A\rarr T1.B \rarr T1.C\rarr T2.A\rarr T2.B\rarr T2.C $$
- Se están ejecutando varias transacciones de forma entrelazada, aunque de manera tal que produzca los mismos resultados que de manera no entrelazada.
    $$ T1.A\rarr T2.A \rarr T1.B\rarr T2.B\rarr T1.C\rarr T2.C $$

### Serialización



Una ejecución concurrente de un conjunto de transacciones es serializable si puede producir el mismo resultado (respecto a los datos, no a la duración) que una ejecución serial de las mismas transacciones.

>**NOTA:**
>
>Hay ejecuciones que nunca serán serializables: **Las Ejecuciones Cíclicas**.
>
>En ellas, ocurre que se genera una dependenica circular donde se imposibilita la reordenación.

### Locking



Es una herramienta que permite bloquear un ítem para su escritura y lectura, como si de una cerradura se tratase.

Existen dos tipos de cierre:
- **Cierre de Lectura o Read Lock**
- **Cierre de Escritura o Write Lock**

El conflicto entre cierres aparece si al menos uno de los implicados es de escritura. Se resuelve generando una zona de exlcusión mutua: Se debe esperar  a que el cierre se libere para que lo sigan tomando.



### Cierre de dos Fases y Deadlock

Una vez que ya conocemos la necesidad de garantizar la serialización de transacciones concurrentes y el concepto de *Locking*, aparece un protocolo que nos garantiza la serialización de cualquier ejecución.




#### Protocolo del Cierre de dos Fases



Consiste en seguir dos reglas simples:

1. Una transacción debe adquirir todos los locks que necesite **sin liberar ninguno**.
2. Una vez que comienza a liberar locks, **no puede adquirir nuevos**.

Esto garantiza la **serialización** de cualquier ejecución, ya que impide que las operaciones de diferentes transacciones se entrelacen de forma que viole el orden serial equivalente.

#### Deadlock

El concepto de *Locking* funciona bien hasta que se presentan situaciones en las que dos o más transacciones se bloquean mutuamente esperando recursos que la otra posee: el famoso interbloqueo o **deadlock**.

**Prevención del Deadlock**  
Para prevenir el deadlock, se utiliza un sistema de **priorización basado en timestamps**.  
Cada transacción tiene un timestamp, y al solicitar un lock:

- Si su timestamp es **anterior**, tiene mayor prioridad y puede continuar, forzando a la otra a abortar o esperar.  
- Si su timestamp es **posterior**, deberá esperar o abortar, evitando así ciclos de espera.


**Detección de Deadlock**

El deadlock puede detectarse representando cada transacción como un nodo en un grafo, donde los conflictos generan enlaces dirigidos. Si se forma un ciclo, entonces hay **deadlock**.